<a href="https://colab.research.google.com/github/mebirtukan/Event_detection/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv1D, MaxPool1D, Conv2D, ConvLSTM2D, MaxPool3D, ConvLSTM3D, LSTM
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, cohen_kappa_score



In [ ]:
!wget www.kasprowski.pl/datasets/events.zip
!unzip events.zip > xx

--2022-07-20 23:29:36--  http://www.kasprowski.pl/datasets/events.zip
Resolving www.kasprowski.pl (www.kasprowski.pl)... 94.152.164.213
Connecting to www.kasprowski.pl (www.kasprowski.pl)|94.152.164.213|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2909538 (2.8M) [application/zip]
Saving to: ‘events.zip’

events.zip          100%[===================>]   2.77M  2.68MB/s    in 1.0s    

2022-07-20 23:29:38 (2.68 MB/s) - ‘events.zip’ saved [2909538/2909538]



In [ ]:
import csv
import numpy as np

def caluculate_velocity(data):
  times = data[:,[0]]
  ts = []

  # convert time to second
  for t in times:
    ts.append(float(t)/1000.0)
  
  Xs = data[:,[0]]
  Ys = data[:,[1]]

  diffX = [] #x values difference
  diffY = [] #y values difference 
  diffT = [] #time values difference

  # difX=x2-x1 and diffY=y2-y1
  for i in range(len(data) - 1):
    diffX.append(float(Xs[i+1]) - float(Xs[i]) )
    diffY.append(float(Ys[i+1]) - float(Ys[i]) )
    diffT.append(float(ts[i+1]) - float(ts[i]) )
  
  # distance = sqrt(x^2 + y^2)
    distancex=diffX
    distancey=diffY
  #distancexx = np.sqrt(np.power(diffX,2) + np.power(diffY,2))
  #distanceyy = np.sqrt(np.power(diffX,2) + np.power(diffY,2))

  #print('distance=', distance)
  #velocityx = np.divide(distancex,diffT)
  #velocityy = np.divide(distancey,diffT)
  velocityx=distancex
  velocityy=distancey

  #velocity_x = []
  #velocity_y=[]

  # calculate velocity
  #vel_X=velocityx
  #vel_Y=velocityy
  #velocity = np.divide(distance,diffT)
  velocityX = np.absolute(velocityx)
  velocityY = np.absolute(velocityy)
  

  #velocity = distance
  #velocity = diffX+diffY
  #velocity=np.absolute((velocityX, velocityY))
  #velocity = np.absolute(velocity)
  #print('velocity=', velocity)
  #print(('minimumu_velocity=',min(velocity)))
  #print(('maximumu_velocity=',max(velocity)))

 # for v in velocity:
 #   print(v,v_threshold,v<v_threshold)
  print('Time=', (diffT))
  print('velocitX=', velocityX)
  print('velocitY=', velocityY)

  #mvmts = []
 
  #store 1 in mvmts[] if velocity is less than threshold else store 2

  return velocityX, velocityY
  #print(velocity_x, velocity_y)





In [ ]:
def openfile(filename):
  mat = loadmat(filename)
  mdata = mat['ETdata']
  mtype = mdata.dtype
  ndata = {n: mdata[n][0,0] for n in mtype.names}
  ndata

  data_headline = ndata['pos']
  print(data_headline)
  data_headline = data_headline[0]
  data_raw = ndata['pos']
  len(data_raw)
  pdata = pd.DataFrame(data_raw,columns=data_headline)
  print(pdata)
  df=pd.DataFrame(pdata)
  df=pd.DataFrame(pdata)
  df.dtypes
  df[1.0]=df[1.0].astype(int)
  df[1.0].dtypes
  x=pdata.iloc[:, 3:5].values
  y=pdata.iloc[:, 5].values
  x[0]
  x.shape[0]
  return x ,y

In [ ]:
#files_to_load = ['data/TH34_img_Europe_labelled_MN.mat','data/TH34_video_BergoDalbana_labelled_MN.mat', 'data/UH27_img_vy_labelled_MN.mat']


In [ ]:
#x,y=openfile('data/TH34_img_Europe_labelled_MN.mat')


In [ ]:
#x1, y1=openfile('data/UH33_img_vy_labelled_MN.mat')


In [ ]:


def open_list_of_files(files_to_load):
  samples = []
  labels =[]
  for my_file in files_to_load:
    sam,lab = openfile(my_file)
    #ssam,slab = make_sequences(sam,lab,50)
    print('Number of samples so far:',len(samples))
    samples.extend(sam)
    labels.extend(lab)
  samples = np.array(samples)
  labels = np.array(labels)
  print('Number of samples at the end:',len(samples))
  return samples,labels
files_to_load = ['data/TH34_img_Europe_labelled_MN.mat','data/TH34_img_Europe_labelled_RA.mat']

s,l = open_list_of_files(files_to_load)
print(s.shape)
print(l.shape)

[[5.78164147e+09 1.90000000e+01 2.00000000e+01 5.22047500e+02
  3.72409700e+02 1.00000000e+00]
 [5.78164347e+09 1.90000000e+01 2.00000000e+01 5.22633900e+02
  3.71951700e+02 1.00000000e+00]
 [5.78164547e+09 1.90000000e+01 2.00000000e+01 5.23043200e+02
  3.72464200e+02 1.00000000e+00]
 ...
 [5.79161349e+09 2.30000000e+01 2.50000000e+01 7.26554300e+02
  6.79770400e+02 1.00000000e+00]
 [5.79161549e+09 2.30000000e+01 2.50000000e+01 7.26536700e+02
  6.79873800e+02 1.00000000e+00]
 [5.79161749e+09 2.30000000e+01 2.50000000e+01 7.26183400e+02
  6.79876200e+02 1.00000000e+00]]
      5.781641e+09  1.900000e+01  2.000000e+01  5.220475e+02  3.724097e+02  \
0     5.781641e+09          19.0          20.0      522.0475      372.4097   
1     5.781643e+09          19.0          20.0      522.6339      371.9517   
2     5.781645e+09          19.0          20.0      523.0432      372.4642   
3     5.781647e+09          19.0          20.0      523.3468      371.7407   
4     5.781649e+09          18.0  

In [ ]:
vx,vy=caluculate_velocity(x)
#vx1,vy1=caluculate_velocity(x1)



print(len(vx))
print(vx[0])
print(np.array(vx).shape)
print(np.array(vy).shape)
#print(len(vx1))
#print(vx1[0])
#print(np.array(vx1).shape)
#print(np.array(vy1).shape)


In [ ]:
y = np.array(y[:-1])
#y1 = np.array(y1[:-1])

In [ ]:
com = np.vstack((vx,vy)).T
#com1 = np.vstack((vx1,vy1)).T


In [ ]:
# sequence 
def make_sequences(samples, labels, sequence_dim = 100, sequence_lag = 1, sequence_attributes = 2):
    nsamples = []
    nlabels = [] 
    for i in range(0,samples.shape[0]-sequence_dim,sequence_lag):
            nsample = np.zeros((sequence_dim,sequence_attributes))
            for j in range(i,i+sequence_dim):
                nsample[j-i,0] = samples[j,0]
                nsample[j-i,1] = samples[j,1]
            nlabel = labels[i+sequence_dim//2]
            #print("Sample",nsample)
            #print("Label",nlabel)
            nsamples.append(nsample)
            nlabels.append(nlabel)
        
    samples = np.array(nsamples)
    labels = np.array(nlabels)
    return samples,labels 

In [ ]:
sequence_dim = 100
print("Samples shape before sequencing",com.shape)

print("Converting to sequences of length {}".format(sequence_dim))
x, y = make_sequences(com, y, sequence_dim)
print("Samples shape after sequencing: {}".format(x.shape))
print("Labels shape after sequencing: {}".format(y.shape))

Samples shape before sequencing (4987, 2)
Converting to sequences of length 100
Samples shape after sequencing: (4887, 100, 2)
Labels shape after sequencing: (4887,)


In [ ]:

#print("Samples shape before sequencing",com1.shape)
#print("Converting to sequences of length {}".format(sequence_dim))
#x1,y1 = make_sequences(com1, y1, sequence_dim)
#print("Samples shape after sequencing: {}".format(x1.shape))
#print("Labels shape after sequencing: {}".format(y1.shape))

In [ ]:
#x,y=openfile('data/UH27_img_vy_labelled_MN.mat')
#x,y=openfile('data/UH33_img_vy_labelled_MN.mat')
#x,y=openfile('data/UH33_img_vy_labelled_MN.mat')


In [ ]:
#x1, y1=openfile('data/TH34_img_Europe_labelled_MN.mat')
#x1,y1=openfile('data/UH33_img_vy_labelled_MN.mat')

#x1, y1=openfile('UH47_video_BergoDalbana_labelled_RA.mat')(test good with 30 MN training)


In [ ]:
y.shape

(4887,)

In [ ]:
#x=np.reshape(x,(4938, 50,2,1))

In [ ]:
#x1=np.reshape(x1,(4938, 50,2,1))

In [ ]:
#x1.shape, x.shape

In [ ]:
from tensorflow.keras.layers import BatchNormalization
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(y)
y = lb.transform(y)
#y1 = lb.transform(y1)


In [ ]:
y.shape[0]

4887

In [ ]:
kf=KFold(n_splits=4)
for train_index, test_index in kf.split(x):
    print(train_index, test_index)

In [ ]:
def get_score(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    return model.score(x_test, y_test)

In [ ]:
get_score(model, x_train, x_test, y_train, y_test)

In [ ]:
#kf=KFold(n_splits=4)
#oos_Y=[]
#oos_pred=[]
#fold=0
#for train, test in kf.split(X):
  #fold+=1
 # print(f'fold #{fold}')
  #x_train=X[train]
  #y_train=Y[train]
  #x_test=X[test]
  #y_test=Y[test]

In [ ]:
model = Sequential()
inputShap=(100,2)
model.add(LSTM(128, input_shape=(inputShap),return_sequences=True))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(64, activation='relu'))
#model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
#model.add(Flatten())

model.add(Dense(64, activation='sigmoid'))
model.add(Dense(y.shape[1], activation='softmax'))
print(inputShap)
model.summary()


(100, 2)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 100, 128)          67072     
                                                                 
 activation_4 (Activation)   (None, 100, 128)          0         
                                                                 
 batch_normalization_8 (Batc  (None, 100, 128)         512       
 hNormalization)                                                 
                                                                 
 dropout_8 (Dropout)         (None, 100, 128)          0         
                                                                 
 lstm_9 (LSTM)               (None, 64)                49408     
                                                                 
 batch_normalization_9 (Batc  (None, 64)               256       
 hNormalization)                             

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test =train_test_split(x, y, test_size=0.25)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy'])


    


In [ ]:
model.fit(x_train, y_train, epochs=15
              ,validation_data=(x_test,y_test)
              )

Epoch 1/15
115/115 [==============================] - 25s 187ms/step - loss: 0.5644 - accuracy: 0.7662 - val_loss: 0.4777 - val_accuracy: 0.8609
Epoch 2/15
115/115 [==============================] - 21s 180ms/step - loss: 0.1970 - accuracy: 0.9220 - val_loss: 0.4536 - val_accuracy: 0.8609
Epoch 3/15
115/115 [==============================] - 21s 185ms/step - loss: 0.1430 - accuracy: 0.9449 - val_loss: 0.3320 - val_accuracy: 0.8609
Epoch 4/15
115/115 [==============================] - 21s 183ms/step - loss: 0.1389 - accuracy: 0.9441 - val_loss: 0.4415 - val_accuracy: 0.8609
Epoch 5/15
115/115 [==============================] - 21s 182ms/step - loss: 0.1355 - accuracy: 0.9476 - val_loss: 0.2441 - val_accuracy: 0.9075
Epoch 6/15
115/115 [==============================] - 21s 183ms/step - loss: 0.1266 - accuracy: 0.9473 - val_loss: 0.1439 - val_accuracy: 0.9378
Epoch 7/15
115/115 [==============================] - 21s 182ms/step - loss: 0.1128 - accuracy: 0.9558 - val_loss: 0.0833 - val_ac

In [ ]:
y1_pred = model.predict(x_test)
print(confusion_matrix(y_test.argmax(axis=1), y1_pred.argmax(axis=1)))
print(classification_report(y_test.argmax(axis=1), y1_pred.argmax(axis=1)))
CM=(confusion_matrix(y_test.argmax(axis=1), y1_pred.argmax(axis=1)))
print("CNN Accuracy: {:.2f}".format(accuracy_score(y_test.argmax(axis=1), y1_pred.argmax(axis=1))))
print("Cohen's Kappa {:.2f}".format(cohen_kappa_score(y_test.argmax(axis=1), y1_pred.argmax(axis=1))))

[[1019   30    3]
 [   6  111    7]
 [   5    2   39]]
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      1052
           1       0.78      0.90      0.83       124
           2       0.80      0.85      0.82        46

    accuracy                           0.96      1222
   macro avg       0.85      0.90      0.88      1222
weighted avg       0.96      0.96      0.96      1222

CNN Accuracy: 0.96
Cohen's Kappa 0.83
